In [ ]:
# this agent makes more use of tools using the @tool decorator and using runtime context.

from langchain.agents import create_agent
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime

SYSTEM_PROMPT = """You are a warehouse manager.

You have access to two tools:

- get_warehouse_status: use this to get warehouse status for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the warehouse status, make sure they are not in the break room. Use the get_user_location tool to find their location."""

@tool
def get_warehouse_status(loc: str) -> str:
    """Get warehouse status for a given location."""
    return f"The warehouse in {loc} is always up and running and ready to ship today except in the countries France and Canada."

@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str


@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve information based on user ID."""
    user_id = runtime.context.user_id
    return "main office" if user_id == "1" else "break room"

agent = create_agent(
    model="claude-haiku-4-5-20251001",
    tools=[get_warehouse_status, get_user_location],
    system_prompt=SYSTEM_PROMPT
)

agent.invoke(
    {"messages": [{"role": "user", "content": "What is the status of the warehouse in Milford, PA?"}]},
    context=Context(user_id="2"),
)